In [12]:
import random
import os
import numpy as np
import torch
import json
from tqdm import tqdm
import pandas as pd
from transformers import RobertaTokenizer
from transformers import AutoTokenizer, AutoModel

In [13]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")

In [15]:
#for getting non-translated data only 
df = df[(df["url1_lang"] == "en") & (df["url2_lang"] == "en")]

In [16]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [17]:
# truncate text from either head or tail part
def trunc_text(text, trunc_pos, length):

    text_ids = tokenizer.encode(text)[1:-1]

    if trunc_pos == 'head':
        text_trunc_ids = text_ids[:length]
    elif trunc_pos == 'tail':
        text_trunc_ids = text_ids[-length:]

    text_trunc_tokens = tokenizer.decode(text_trunc_ids)
    #text_trunc_back_sent = ''.join([x.replace('_', ' ') for x in text_trunc_tokens])[:-1]

    return text_trunc_tokens

In [18]:
trunc_text(df["text1"][0], "tail", 20)

'reserved. this material may not be published, broadcast, rewritten or redistributed.'

In [21]:
#following columns are for non-translated data
df = df.dropna(subset=["ogTitle1", "ogTitle2", "ogText1", "ogText2"])

In [22]:
df["titleText1"] = df["ogTitle1"] + df["ogText1"]
df["titleText2"] = df["ogTitle2"] + df["ogText2"]

<ipython-input-22-ca814b37620b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["titleText1"] = df["ogTitle1"] + df["ogText1"]
<ipython-input-22-ca814b37620b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["titleText2"] = df["ogTitle2"] + df["ogText2"]


In [6]:
"""
#for when working with translated data
df = df.dropna(subset=["title1", "title2", "text1", "text2"])
df["titleText1"] = df["title1"] + df["text1"]
df["titleText2"] = df["title2"] + df["text2"]
"""

In [23]:
HEAD_COUNT = 288
TAIL_COUNT = 96

In [24]:
df["text1Head"] = df["titleText1"] .apply(trunc_text, args=["head", HEAD_COUNT])
df["text1Tail"] = df["text1"].apply(trunc_text, args=["tail", TAIL_COUNT])

df["text2Head"] = df["titleText2"].apply(trunc_text, args=["head", HEAD_COUNT])
df["text2Tail"] = df["text2"].apply(trunc_text, args=["tail", TAIL_COUNT])

Token indices sequence length is longer than the specified maximum sequence length for this model (708 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-24-32459779fe44>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text1Head"] = df["titleText1"] .apply(trunc_text, args=["head", HEAD_COUNT])
<ipython-input-24-32459779fe44>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text1Tail"] = df["text1"].apply(trunc_text, args=["tail", TAIL_COUNT])
<ipython-inp

In [25]:
df["text1Merged"] = df["text1Head"] + df["text1Tail"]
df["text2Merged"] = df["text2Head"] + df["text2Tail"]

<ipython-input-25-7b4fa12297d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text1Merged"] = df["text1Head"] + df["text1Tail"]
<ipython-input-25-7b4fa12297d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text2Merged"] = df["text2Head"] + df["text2Tail"]


In [27]:
df.to_csv("/home/blitt/projects/localNews/data/processed/en_288_96.tsv", sep="\t")